# quality_assurance

> In progress, including development of diagnostics to be moved elsewhere when complete.

In [ ]:
from abc import ABC, abstractmethod

import dask
import pandas as pd
import xarray as xr

from qagmire.data import (
    get_lr_l2_stack_files,
    read_class_spec,
    read_class_table,
    read_galaxy_table,
)

## Diagnostics

Now let's look at some diagnostic tests.

### Line flux check

A reproduction of the weaveio [line_flux_check](https://github.com/bamford/QAG/blob/master/diagnostics/line_flux_check.py).

In [ ]:
def line_wavelengths(galaxy_table, class_table):
    """Determine the expected observed wavelengths of all lines."""
    lines = galaxy_table["LINE"].astype(str)
    line_species, line_rest_wl = lines.str.split(dim="", sep="_").T
    line_rest_wl = line_rest_wl.astype(float)
    line_wl = (1 + class_table["Z"]) * line_rest_wl
    return line_wl

In [ ]:
def wavelength_boundaries(class_spec):
    """Determine wavelength boundaries and wavelength gaps of blue and red spectra."""
    gaps = {}
    boundaries = {}
    for band, low, high in (("B", 4000, 6000), ("R", 6000, 9000)):
        wl_dim = f"LAMBDA_{band}"
        wl = class_spec[wl_dim]
        null_flux = class_spec[f"FLUX_RR_{band}"].isnull()
        wl_null = wl.where(null_flux & (wl > low) & (wl < high))
        gaps[band] = {"low": wl_null.min(dim=wl_dim), "high": wl_null.max(dim=wl_dim)}
        wl_not_null = wl.where(~null_flux)
        boundaries[band] = {
            "low": wl_not_null.min(dim=wl_dim),
            "high": wl_not_null.max(dim=wl_dim),
        }
    return boundaries, gaps

In [ ]:
class Diagnostics(ABC):
    def run(self):
        tests = self.tests()
        test_names = [t["name"] for t in tests]
        test_array = [t["test"] for t in tests]
        detail = xr.concat(test_array, pd.Index(test_names, name="test"))
        detail = dask.compute(detail)[0]
        return detail

    @abstractmethod
    def tests(self, **kwargs):
        return [
            {
                "name": "a_short_name",
                "description": "The question that the test answers",
                "test": None,
            },  # a boolean DataArray including dimensions `filename` and `APS_ID`
        ]

In [ ]:
class LineFluxCheck(Diagnostics):
    def tests(self, **kwargs):
        lr_l2_stack_files = get_lr_l2_stack_files(**kwargs)

        class_spec = read_class_spec(lr_l2_stack_files)
        galaxy_table = read_galaxy_table(lr_l2_stack_files)
        class_table = read_class_table(lr_l2_stack_files)

        line_wl = line_wavelengths(galaxy_table, class_table)
        boundaries, gaps = wavelength_boundaries(class_spec)

        measured_line_flux = galaxy_table["LINES"].sel(QTY="FLUX", drop=True)
        null_flux = measured_line_flux.isnull()

        is_in_red_gap = (line_wl > gaps["R"]["low"]) & (line_wl < gaps["R"]["high"])
        is_in_blue_gap = (line_wl > gaps["B"]["low"]) & (line_wl < gaps["B"]["high"])
        is_in_gap = is_in_blue_gap | is_in_red_gap

        is_off_spectrum = (
            (line_wl < boundaries["B"]["low"]) | (line_wl > boundaries["B"]["high"])
        ) & ((line_wl < boundaries["R"]["low"]) | (line_wl > boundaries["R"]["high"]))

        is_on_spectrum = ~is_in_gap & ~is_off_spectrum

        null_spectrum = (
            boundaries["B"]["low"].isnull() | boundaries["R"]["low"].isnull()
        )

        tests = [
            {
                "name": "line_in_null_spectrum",
                "description": "Do non-null line fluxes appear in completely null spectra?",
                "test": ~null_flux & null_spectrum,
            },
            {
                "name": "line_in_blue_chip_gap",
                "description": "Do non-null line fluxes appear in the blue chip gap?",
                "test": ~null_flux & is_in_blue_gap,
            },
            {
                "name": "line_in_red_chip_gap",
                "description": "Do non-null line fluxes appear in the red chip gap?",
                "test": ~null_flux & is_in_red_gap,
            },
            {
                "name": "line_off_spectrum",
                "description": "Do non-null line fluxes appear outside the observed wavelength range?",
                "test": ~null_flux & is_off_spectrum,
            },
            {
                "name": "null_line_on_spectrum",
                "description": "Do null line fluxes appear in an observed wavelength range?",
                "test": null_flux & is_on_spectrum,
            },
        ]
        return tests

In [ ]:
detail = LineFluxCheck().run()

Locating and converting where necessary: 100%|██████████| 17/17 [00:00<00:00, 11491.24it/s]
Reading netCDF files... took 2.85 s.
Locating and converting where necessary: 100%|██████████| 17/17 [00:00<00:00, 18088.07it/s]
Reading netCDF files... took 2.54 s.
Locating and converting where necessary: 100%|██████████| 17/17 [00:00<00:00, 16070.13it/s]
Reading netCDF files... took 2.73 s.


/home2/bamford/anaconda/envs/qagmire/lib/python3.12/site-packages/dask/array/reductions.py:621: RuntimeWarning: All-NaN slice encountered
  return np.nanmin(x_chunk, axis=axis, keepdims=keepdims)
/home2/bamford/anaconda/envs/qagmire/lib/python3.12/site-packages/dask/array/reductions.py:621: RuntimeWarning: All-NaN slice encountered
  return np.nanmin(x_chunk, axis=axis, keepdims=keepdims)


In [ ]:
detail = detail.swap_dims({"filename": "obid"}).drop_vars("filename")

In [ ]:
def fails_summary(da, top=None):
    df = da.to_dataframe(name="fails").unstack("test")
    df.loc[:, ("fails", "total")] = df.sum(axis="columns")
    df = df.sort_values(("fails", "total"), ascending=False)
    if top is not None:
        df = df.iloc[:top]
    return df

In [ ]:
per_obid = detail.sum(dim=["APS_ID", "LINE"])
fails_summary(per_obid)

fails                                             \
test line_in_null_spectrum line_in_blue_chip_gap line_in_red_chip_gap   
obid                                                                    
3170                     0                    15                   37   
3189                     0                    20                   26   
3133                     0                    10                   22   
3191                     0                     5                    7   
3175                     0                    52                   44   
3380                     0                    32                   46   
3434                     0                    37                   63   
3346                     0                    22                   94   
3217                     0                    37                   87   
3372                     0                   144                  178   
3295                     0                   141                  200   
3756                     0                   161                  194   
3900                     0                   162                  226   
3653                     0                   165                  196   
3802                     0                   158                  185   
3806                     0                   165                  202   
3803                     0                   164                  195   

                                                     
test line_off_spectrum null_line_on_spectrum  total  
obid                                                 
3170               261                 17500  17813  
3189               267                 17481  17794  
3133               288                 17428  17748  
3191                97                 17226  17335  
3175               709                 12132  12937  
3380               674                 12159  12911  
3434               747                 11182  12029  
3346              1684                  6496   8296  
3217              1781                  6353   8258  
3372              2695                  3727   6744  
3295              2541                  3719   6601  
3756              2651                  3575   6581  
3900              2646                  3536   6570  
3653              2558                  3544   6463  
3802              2508                  3466   6317  
3806              2497                  3453   6317  
3803              2519                  3418   6296

In [ ]:
per_line = detail.sum(dim=["obid", "APS_ID"])
fails_summary(per_line)

fails                        \
test            line_in_null_spectrum line_in_blue_chip_gap   
LINE                                                          
[ArIII]_7135.67                     0                     0   
[SII2]_6730.68                      0                     0   
[SII]_6716.31                       0                     0   
[NII]_6583.34                       0                     0   
Ha_6562.80                          0                     0   
[OI]_6300.20                        0                     0   
[OII]_3726.03                       0                   178   
HeI_5875.60                         0                     1   
[OII]_3728.73                       0                   172   
[NeIII]_3967.40                     0                    81   
[NeIII]_3868.69                     0                   103   
[OIII]_4363.15                      0                   136   
[NI]_5200.39                        0                    51   
[NI]_5197.90                        0                    51   
HeII_4685.74                        0                    84   
[ArIV]_4711.30                      0                    84   
[OIII]_5006.77                      0                    45   
[ArIV]_4740.10                      0                    50   
Hb_4861.32                          0                    44   
HeII_3203.15                        0                   160   
[NeV]_3345.81                       0                   135   
[NeV]_3425.81                       0                   115   

                                                                              \
test            line_in_red_chip_gap line_off_spectrum null_line_on_spectrum   
LINE                                                                           
[ArIII]_7135.67                   55              4343                  6604   
[SII2]_6730.68                    53              3583                  6626   
[SII]_6716.31                     51              3576                  6625   
[NII]_6583.34                     47              3361                  6634   
Ha_6562.80                        52              3313                  6633   
[OI]_6300.20                      73              2776                  6638   
[OII]_3726.03                      0                 0                  8491   
HeI_5875.60                      136              1837                  6655   
[OII]_3728.73                      0                 0                  8454   
[NeIII]_3967.40                  229                 0                  8185   
[NeIII]_3868.69                    0                 0                  8248   
[OIII]_4363.15                   137                 0                  7520   
[NI]_5200.39                     115               569                  6662   
[NI]_5197.90                     118               566                  6662   
HeII_4685.74                     191                 0                  6957   
[ArIV]_4711.30                   211                 0                  6904   
[OIII]_5006.77                   182               200                  6659   
[ArIV]_4740.10                   158                 0                  6867   
Hb_4861.32                       194                 0                  6736   
HeII_3203.15                       0              1159                  3888   
[NeV]_3345.81                      0               969                  3884   
[NeV]_3425.81                      0               871                  3863   

                        
test             total  
LINE                    
[ArIII]_7135.67  11002  
[SII2]_6730.68   10262  
[SII]_6716.31    10252  
[NII]_6583.34    10042  
Ha_6562.80        9998  
[OI]_6300.20      9487  
[OII]_3726.03     8669  
HeI_5875.60       8629  
[OII]_3728.73     8626  
[NeIII]_3967.40   8495  
[NeIII]_3868.69   8351  
[OIII]_4363.15    7793  
[NI]_5200.39      7397  
[NI]_5197.90      7397  
HeII_4685.74      7232  
[ArIV

In [ ]:
per_fibre = detail.sum(dim=["obid", "LINE"])
fails_summary(per_fibre, top=20)

fails                                             \
test   line_in_null_spectrum line_in_blue_chip_gap line_in_red_chip_gap   
APS_ID                                                                    
628                        0                     0                    0   
629                        0                     0                    0   
341                        0                     0                    0   
144                        0                     0                    0   
436                        0                     2                    2   
722                        0                     0                    0   
437                        0                     6                    0   
434                        0                     6                    0   
242                        0                     0                    0   
339                        0                     6                    0   
52                         0                     0                    1   
243                        0                     0                    0   
48                         0                     1                    0   
530                        0                     0                    6   
338                        0                     0                    0   
821                        0                     0                    0   
817                        0                     0                    0   
49                         0                     3                    0   
627                        0                     2                    6   
128                        0                     1                    0   

                                                      
test   line_off_spectrum null_line_on_spectrum total  
APS_ID                                                
628                    6                   343   349  
629                    3                   340   343  
341                    0                   332   332  
144                    3                   324   327  
436                   12                   310   326  
722                   14                   304   318  
437                   15                   296   317  
434                   15                   296   317  
242                   21                   280   301  
339                   18                   277   301  
52                     9                   290   300  
243                   21                   277   298  
48                     1                   296   298  
530                   12                   279   297  
338                   21                   274   295  
821                   12                   281   293  
817                    7                   283   290  
49                    24                   262   289  
627                   18                   261   287  
128                   21                   264   286